# Eat Safe, Love

## Part 1: Database and Jupyter Notebook Set Up

Import the data provided in the `establishments.json` file from your Terminal. Name the database `uk_food` and the collection `establishments`.

Within this markdown cell, copy the line of text you used to import the data from your Terminal. This way, future analysts will be able to repeat your process.

e.g.: Import the dataset with `mongoimport --type json -d uk_food -c establishments --drop --jsonArray establishments.json`

In [2]:
# Import dependencies
from pymongo import MongoClient
from pprint import pprint

In [4]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [6]:
# confirm that our new database was created
print(mongo.list_database_names())

['admin', 'config', 'local', 'met', 'uk_food']


In [8]:
# assign the uk_food database to a variable name
db = mongo['uk_food']

In [10]:
# review the collections in our new database
print(db.list_collection_names())

['establishments']


In [12]:
# review a document in the establishments collection
pprint(db.establishments.find_one())

{'AddressLine1': 'The Pines Garden',
 'AddressLine2': 'Beach Road',
 'AddressLine3': 'St Margarets Bay',
 'AddressLine4': 'Kent',
 'BusinessName': 'The Pines Calyx',
 'BusinessType': 'Other catering premises',
 'BusinessTypeID': 7841,
 'ChangesByServerID': 0,
 'Distance': 4587.362402580997,
 'FHRSID': 254250,
 'LocalAuthorityBusinessID': 'PI/000066174',
 'LocalAuthorityCode': '182',
 'LocalAuthorityEmailAddress': 'publicprotection@dover.gov.uk',
 'LocalAuthorityName': 'Dover',
 'LocalAuthorityWebSite': 'http://www.dover.gov.uk/',
 'NewRatingPending': False,
 'Phone': '',
 'PostCode': 'CT15 6DZ',
 'RatingDate': '2021-08-17T00:00:00',
 'RatingKey': 'fhrs_5_en-gb',
 'RatingValue': '5',
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('67214e8f73b07a3c3b43e99d'),
 'geocode': {'latitude': '51.148133', 'longitude': '1.383298'},
 'links': [{'href': 'https://api.ratings.food.gov.uk/establishments/254250',
            'rel': 'self'}],
 'meta': {'dataSource': None,
          'extract

In [14]:
# assign the collection to a variable
establishments = db['establishments']

## Part 2: Update the Database

1. An exciting new halal restaurant just opened in Greenwich, but hasn't been rated yet. The magazine has asked you to include it in your analysis. Add the following restaurant "Penang Flavours" to the database.

In [16]:
# Create a dictionary for the new restaurant data
penang_flavours = {
    "BusinessName":"Penang Flavours",
    "BusinessType":"Restaurant/Cafe/Canteen",
    "BusinessTypeID":"",
    "AddressLine1":"Penang Flavours",
    "AddressLine2":"146A Plumstead Rd",
    "AddressLine3":"London",
    "AddressLine4":"",
    "PostCode":"SE18 7DY",
    "Phone":"",
    "LocalAuthorityCode":"511",
    "LocalAuthorityName":"Greenwich",
    "LocalAuthorityWebSite":"http://www.royalgreenwich.gov.uk",
    "LocalAuthorityEmailAddress":"health@royalgreenwich.gov.uk",
    "scores":{
        "Hygiene":"",
        "Structural":"",
        "ConfidenceInManagement":""
    },
    "SchemeType":"FHRS",
    "geocode":{
        "longitude":"0.08384000",
        "latitude":"51.49014200"
    },
    "RightToReply":"",
    "Distance":4623.9723280747176,
    "NewRatingPending":True
}

In [18]:
# Insert the new restaurant into the collection
establishments.insert_one(penang_flavours)

InsertOneResult(ObjectId('67215267917ddb43f28891e7'), acknowledged=True)

In [22]:
# Check that the new restaurant was inserted
# Query the collection to find the new restaurant
result = establishments.find_one({"BusinessName": "Penang Flavours"})

# Check if the restaurant was found and print the result
if result:
    print("Restaurant found")
else:
    print("Restaurant not found.")

Restaurant found


2. Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the `BusinessTypeID` and `BusinessType` fields.

In [36]:
# Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the BusinessTypeID and BusinessType fields
query = {'BusinessType': 'Restaurant/Cafe/Canteen'}
fields = {"BusinessTypeID": 1, "BusinessType": 1, "_id": 0}
results = establishments.find_one(query, fields)

#print results:
pprint(results)

{'BusinessType': 'Restaurant/Cafe/Canteen', 'BusinessTypeID': 1}


3. Update the new restaurant with the `BusinessTypeID` you found.

In [40]:
# Update the new restaurant with the correct BusinessTypeID
query = {"BusinessName": "Penang Flavours"}
fields = {"$set": {"BusinessTypeID": 1}}

# Perform the update operation
establishments.update_one(query, fields)

UpdateResult({'n': 1, 'nModified': 1, 'ok': 1.0, 'updatedExisting': True}, acknowledged=True)

In [42]:
# Confirm that the new restaurant was updated
query = {'BusinessName': 'Penang Flavours'}
fields = {"BusinessTypeID": 1, "BusinessType": 1, "_id": 0}
updated_result = establishments.find_one(query, fields)

# Print the updated result to confirm the update
pprint(updated_result)

{'BusinessType': 'Restaurant/Cafe/Canteen', 'BusinessTypeID': 1}


4. The magazine is not interested in any establishments in Dover, so check how many documents contain the Dover Local Authority. Then, remove any establishments within the Dover Local Authority from the database, and check the number of documents to ensure they were deleted.

In [44]:
# Find how many documents have LocalAuthorityName as "Dover"
query = {'LocalAuthorityName': 'Dover'}
count = establishments.count_documents(query)
print(f"Number of documents with LocalAuthorityName as 'Dover': {count}")

Number of documents with LocalAuthorityName as 'Dover': 994


In [46]:
# Delete all documents where LocalAuthorityName is "Dover"
query = {'LocalAuthorityName': 'Dover'}
result = establishments.delete_many(query)


In [48]:
# Print the result to confirm the number of documents deleted
print(f"Number of documents deleted: {result.deleted_count}")

Number of documents deleted: 994


In [50]:
# Check if any remaining documents include Dover
# Query to find any remaining documents with LocalAuthorityName as "Dover"
query = {'LocalAuthorityName': 'Dover'}
remaining_documents = list(establishments.find(query))

# Check if any documents were found and print the result
if len(remaining_documents) > 0:
    print("There are still documents with LocalAuthorityName as 'Dover'.")
else:
    print("No remaining documents with LocalAuthorityName as 'Dover'.")

No remaining documents with LocalAuthorityName as 'Dover'.


In [54]:
# Check that other documents remain with 'find_one'
# Query to find any document that does not have LocalAuthorityName as "Dover"
query = {'LocalAuthorityName': {'$ne': 'Dover'}}
other_document = establishments.find_one(query)

# Check if a document was found and print the result

pprint(other_document)

{'AddressLine1': 'East Cliff Pavilion',
 'AddressLine2': 'Wear Bay Road',
 'AddressLine3': 'Folkestone',
 'AddressLine4': 'Kent',
 'BusinessName': 'The Pavilion',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 'ChangesByServerID': 0,
 'Distance': 4591.765489457773,
 'FHRSID': 1043695,
 'LocalAuthorityBusinessID': 'PI/000073616',
 'LocalAuthorityCode': '188',
 'LocalAuthorityEmailAddress': 'foodteam@folkestone-hythe.gov.uk',
 'LocalAuthorityName': 'Folkestone and Hythe',
 'LocalAuthorityWebSite': 'http://www.folkestone-hythe.gov.uk',
 'NewRatingPending': False,
 'Phone': '',
 'PostCode': 'CT19 6BL',
 'RatingDate': '2018-04-04T00:00:00',
 'RatingKey': 'fhrs_5_en-gb',
 'RatingValue': '5',
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('67214e8f73b07a3c3b43ec81'),
 'geocode': {'latitude': '51.083812', 'longitude': '1.195625'},
 'links': [{'href': 'https://api.ratings.food.gov.uk/establishments/1043695',
            'rel': 'self'}],
 'meta': {'dataSource': 

5. Some of the number values are stored as strings, when they should be stored as numbers.

Use `update_many` to convert `latitude` and `longitude` to decimal numbers.

In [56]:
# Change the data type from String to Decimal for longitude and latitude
establishments.update_many({},   [
        {'$set': {
                'geocode.longitude': {'$toDouble': '$geocode.longitude'},
                'geocode.latitude': {'$toDouble': '$geocode.latitude'}}}])

UpdateResult({'n': 38786, 'nModified': 38786, 'ok': 1.0, 'updatedExisting': True}, acknowledged=True)

Use `update_many` to convert `RatingValue` to integer numbers.

In [59]:
# Set non 1-5 Rating Values to Null
non_ratings = ["AwaitingInspection", "Awaiting Inspection", "AwaitingPublication", "Pass", "Exempt"]
establishments.update_many({"RatingValue": {"$in": non_ratings}}, [ {'$set':{ "RatingValue" : None}} ])

UpdateResult({'n': 4091, 'nModified': 4091, 'ok': 1.0, 'updatedExisting': True}, acknowledged=True)

In [61]:
# Change the data type from String to Integer for RatingValue
establishments.update_many({}, [{"$set":{"RatingValue": {"$toInt": "$RatingValue"}}}])

UpdateResult({'n': 38786, 'nModified': 34695, 'ok': 1.0, 'updatedExisting': True}, acknowledged=True)

In [65]:
# Check that the coordinates and rating value are now numbers

for doc in establishments.find():
    rating_value = doc.get("RatingValue")
    latitude = doc.get("geocode.latitude")
    longitude = doc.get("geocode.longitude")
     # Check if RatingValue is a number
    if isinstance(rating_value, (int, float)):
        print(f"RatingValue: {rating_value} is a number.")
    else:
        print(f"RatingValue: {rating_value} is NOT a number.")

    # Check if coordinates are numbers
    if isinstance(latitude, (int, float)) and isinstance(longitude, (int, float)):
        print(f"Coordinates: Latitude: {latitude}, Longitude: {longitude} are numbers.")
    else:
        print(f"Coordinates: Latitude: {latitude}, Longitude: {longitude} are NOT numbers.")

RatingValue: 5 is a number.
Coordinates: Latitude: None, Longitude: None are NOT numbers.
RatingValue: 5 is a number.
Coordinates: Latitude: None, Longitude: None are NOT numbers.
RatingValue: 4 is a number.
Coordinates: Latitude: None, Longitude: None are NOT numbers.
RatingValue: 5 is a number.
Coordinates: Latitude: None, Longitude: None are NOT numbers.
RatingValue: 5 is a number.
Coordinates: Latitude: None, Longitude: None are NOT numbers.
